# Klasifikasi dengan Ringkasan

## Import Library



*   nltk : untuk menjalankan preprosesing dokumen.
*   TfidfVectorizer : untuk mengekstrak kalimat menjadi nilai TF-IDF.
*   cosine_similarity : untuk menghitung nilai cosine similarity dari hasil TF-IDF.
*   numpy : untuk menjalankan semua proses yang berhubungan dengan array.
*   re : Regular expression adalah pola teks yang dapat digunakan untuk mencari dan mengganti teks dalam string.
*   networkx : untuk membuat graph.
* sent_tokenize : untuk mengubah dokumen menjadi kalimat-kalimat.







In [ ]:
import nltk
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import re
import networkx as nx
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Load Data

Data diperoleh dari scraping pada berita suararakyat.id yang sudah dilakukan sebelumnya.

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/ppw/tugas/tugas_ppw/suararakyat.csv")
data

,Judul,Isi,Kategori
0,"Hadiri Seminar Nasional, Slamet Ariyadi Gemble...","PAMEKASAN,Suararakyat.id– Slamet Ariyadi luang...",Pendidikan
1,SIAP !! Cerdaskan Siswa-Siswi SD Negeri Giliba...,Mahasiswa UTM adakan Gerakan Memasyarakatkan M...,Pendidikan
2,Direktur Pasca Sarjana IAIN Madura Ajak Mahasi...,"PAMEKASAN,Suararakyat.id– Kegiatan Kuliah Peng...",Pendidikan
3,MK Tolak Gugatan Usia Minimal Capres-Cawapres ...,"JAKARTA, Suararakyat.id–Mahkamah Konstitusi (M...",Politik
4,Sukes Gelar Festival Nasyid Islami :Al-Amin Al...,"SUMENEP,Suararakyat.id– Dalam Rangka Memeriahk...",Pendidikan
...,...,...,...
140,MK Tolak Gugatan Usia Minimal Capres-Cawapres ...,"JAKARTA, Suararakyat.id–Mahkamah Konstitusi (M...",Politik
141,BEM SI Akan Gelar Aksi Demo Besar-Besaran pada...,"JAKARTA, Suararakyat.id– Badan Eksekutif Mahas...",Politik
142,Sportif Cup 2022 : Tim Voli Remaja BPRS siap T...,"SUMENEP,Suararakyat.id– Tim kuat Remaja BPRS a...",Sport
143,"Bupati Sumenep Buka JJC, Minta Guru Memberikan...","SUMENEP,Suararakyat.id– Dinas Pendidikan (Disd...",Pendidikan


## Preprocessing

re.sub(r'[^\w\s.?!,/]', '', input_string) akan menghapus semua tanda baca kecuali karakter alfanumerik, spasi, titik, tanda tanya, koma, dan garis miring.

In [ ]:
#Remove Puncutuation
clean_symbol = re.compile('[^\w\s.?!,/]')
def clean_punct(text):
    text = clean_symbol.sub(' ', text).lower()
    return text
# Buat kolom tambahan untuk data description yang telah diremovepunctuation
preprocessing = data['Isi'].apply(clean_punct)
clean=pd.DataFrame(preprocessing)
clean

,Isi
0,"pamekasan,suararakyat.id slamet ariyadi luang..."
1,mahasiswa utm adakan gerakan memasyarakatkan m...
2,"pamekasan,suararakyat.id kegiatan kuliah peng..."
3,"jakarta, suararakyat.id mahkamah konstitusi m..."
4,"sumenep,suararakyat.id dalam rangka memeriahk..."
...,...
140,"jakarta, suararakyat.id mahkamah konstitusi m..."
141,"jakarta, suararakyat.id badan eksekutif mahas..."
142,"sumenep,suararakyat.id tim kuat remaja bprs a..."
143,"sumenep,suararakyat.id dinas pendidikan disd..."


## Ringkasan

langkah meringkas :
*   Lakukan looping pada dataframe yang akan dilakukan ringkasan
*   Setiap dokumen akan dilakukan :
1.   ektrak dokumen menjadi Kalimat
2.   Ekstrak menjadi matrix TF-IDF
3.   Dilakukan perhitungan cosine similarity antar kalimat
4.   Hasil cosine similarity dbentuk menjadi graph dan dihitung nilai closenes centralitynya.
5.   Hasil closenes centrality di urutkan dari terbesar ke yang terkecil dan dilakukan penggabungan kalimat berdasarkan 3 nilai closenes terbesar.
6.  Kalimat hasil rangkuman disimpan dalam variabel hasil.
*   Proses diatas akan dilakukan terus menerus pada setiap dokumen
*   Hasil dari proses ringkasan akan dirubah kedalam bentuk dataframe untuk digunakan dalam proses modelling









In [ ]:
hasil=[]
for idx,row in clean.iterrows():
    sentences = sent_tokenize(row['Isi'])
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(sentences)
    cos_sim = cosine_similarity(tfidf_matrix)
    threshold=0.01
    G = nx.Graph()
    for i in range(len(sentences)):
        for j in range(len(sentences)):
          sim = cos_sim[i][j]
          if sim > threshold:
            G.add_edge(i, j, weight=sim)
    closeness_centrality = nx.closeness_centrality(G)
    sorted_d = sorted(closeness_centrality.items(), key=lambda x: x[1], reverse=True)
    hasil_sum=[]
    for node, closeness in sorted_d[:3]:
      top_sentence = sentences[node]
      hasil_sum.append(top_sentence)
    hasil.append(''.join(hasil_sum))

In [ ]:
ringkasan= pd.DataFrame(hasil,columns=['Ringkasan'])

In [ ]:
ringkasan

,Ringkasan
0,"pamekasan,suararakyat.id slamet ariyadi luang..."
1,mahasiswa utm adakan gerakan memasyarakatkan m...
2,"m., pd, selaku direktur pasca sarjana iain ma..."
3,"jakarta, suararakyat.id mahkamah konstitusi m..."
4,"sumenep,suararakyat.id dalam rangka memeriahk..."
...,...
140,"jakarta, suararakyat.id mahkamah konstitusi m..."
141,"jakarta, suararakyat.id badan eksekutif mahas..."
142,sementara lybas juga berhasil menaklukkan tim ...
143,"sumenep,suararakyat.id dinas pendidikan disd..."


## Modelling

Dalam proses modelling ini menggunakan metode Naive Bayes, langkah-langkahnya sebagai berikut :
*   Data hasil ringkasan dilakukan proses TF-IDF
*   Hasil TF-IDF dilakukan proses pembangian data training dan data testing dengan perbandingan 80:20.
*   Kemudian data training dilatih dalam model dan data testing digunakan untuk menentukan akurasi.
*   Dari metode Vaive Bayes diperoleh nilai akurasi sebesar 0,9655.




In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

In [ ]:
# Menghitung TF-IDF
vec = TfidfVectorizer()
tfidf_result = vec.fit_transform(ringkasan['Ringkasan']).toarray()

In [ ]:
y=data['Kategori']
X_train,X_test,y_train,y_test = train_test_split(tfidf_result,y,test_size=0.2,random_state=42)

In [ ]:
NB = GaussianNB()
gaus=NB.fit(X_train, y_train)

In [ ]:
y_pred = gaus.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Akurasi: {accuracy}')

Akurasi: 0.9655172413793104
